In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from utils import set_seed
from plot_utils import plot_eval_results
from converters import Converter
from config import cfg
from c_utils import write_model_h
from utils import save_test_data
import plot_utils as pu
from model import save_pruned_model


In [4]:
from tensorflow_datasets.image.nyu_depth_v2 import NyuDepthV2

In [ ]:
NyuDepthV2()

In [9]:
from nyuv2_torch_ds_adapter import get_tf_nyuv2_ds
import argparse
args = argparse.Namespace()
args.truncate_testset = False
args.crop_size = (640, 480)
args.target_size = cfg.img_size
args.out_fold_ratio = 1
args.is_maxim = False
cfg.do_overfit=False
args.batch_size=cfg.batch_size*4

ds_train, ds_val, ds_test = get_tf_nyuv2_ds(cfg.base_dataset_dir, args)

x_val= next(iter(ds_val))
x_train= next(iter(ds_train))



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\heavy\\Documents\\NUYV2-Depth\\nyu2_train.csv'